In [ ]:
from spacy import displacy
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline
import json
import random
import torch
import warnings
warnings.filterwarnings("ignore")

hex_colors = ['#f5cac3',
             '#ffb3ba',
             '#ffdfba',
             '#ffffba',
             '#baffc9',
             '#bae1ff',
             '#bac1ff',
             '#ffbaff',
             '#d6a5c9',
             '#ffa5d8',
             '#ffbfa5',
             '#a5ffd6',
             '#c9a5ff',
             '#a5d6ff',
             '#d6a5ff',
             '#a3ffa3',
             '#c3a5ff',
             '#ffbebe',
             '#c3ffc3']

hex_colors = ['#ED143B', '#100943', '#123AED', '#9A13ED', '#12CCED', '#E4F4F6']

In [ ]:
def load_labels(checkpoint_path):
    with open(f'{checkpoint_path}/config.json', 'r') as f:
        j = json.load(f)
    label_list = list(j['label2id'].keys())
    return label_list

def display_ents(text):
    pred_entities = pipe(text)
    rewrite_pred_entities = []
    for row in pred_entities:
        rewrite_pred_entities.append({'start': row['start'], 'end': row['end'], 'label': row['entity_group']})
    tags = [label.strip('B-') for label in label_list if label.startswith('B-')]
    displacy_dict = [{'text': text, 'ents': rewrite_pred_entities, 'title': None}]
    colors = {ent: hex_colors[i//2-1] for i, ent in enumerate(tags)}
    options = {"ents": tags, "colors": colors}
    html = displacy.render(displacy_dict, style="ent", manual=True, options=options)

In [ ]:
model_checkpoint = '../ner/checkpoint-1446'

label_list = load_labels(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='cpu')
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cpu')

In [ ]:
text = "<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Русский тизер=трейлер (2021) <ОПИСАНИЕ:>Лучший Telegram канал о кино <LINK> Сотрудничество <LINK> Дата выхода 26 августа 2021 Оригинальное название: OSS 117: Alerte rouge en Afrique noire Страна: Франция Режиссер: Николя Бедос Жанр: боевик, комедия В главных ролях: Жан Дюжарден, Пьер Нинэ, Мелоди Каста, Наташа Линдинжер, Владимир Иорданов, Фату Н’Диайе, Пол Уайт Мир изменился. Он нет. Судьба заносит легендарного Агента 117 в Африку, где горячее пустыни только женщины. Вооруженный неиссякаемой уверенностью в себе и убийственной харизмой, он может справиться со всеми врагами, кроме самого себя. По вопросам авторского права, пожалуйста, свяжитесь с нами по адресу: <AT>"

In [ ]:
display_ents(text)